In [175]:
# FIXME: outdated code
assert False

# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
import os
import sys
from pathlib import Path

# add path to domblar development code
sys.path.insert(0, str(Path(os.getcwd()).parent))

synth_count = 4

from domblar.domblar import Domblar
d = Domblar(
    synth_count=synth_count,
    context='dexed',
)

/Users/gexahedron/devel


In [176]:
import random
from copy import copy, deepcopy

from domblar.edo import note_in_scale

from domblar.music_math import ratio_to_cents, OCTAVE_IN_CENTS


# FIXME: what to do with defaults here? should i remove them?
# def cycle(a, dup_start=False, dup_end=False):
#     end = -2
#     if dup_end:
#         end = -2
#     if dup_start:
#         return a + a[end::-1]
#     else:
#         return a + a[end:0:-1]


def sequence(start, ops, mod, notes):
    result = [start]
    idx = 0
    while len(result) != notes:
        prev = result[-1]
        new = prev + ops[idx]
        while new <= 0:
            new += mod
        while new > mod:
            new -= mod
        result.append(new)
        idx = (idx + 1) % len(ops)
    return result


def build_scale_from_steps(steps, edo=None, start=0):
    if type(steps) is str:
        steps = [int(x) for x in steps]
    assert type(steps) is list
    scale = [start]
    for step in steps[:-1]:
        scale.append(scale[-1] + step)
    if edo is not None:
        assert scale[-1] + steps[-1] == scale[0] + edo
    return scale


def cents_to_steps(cents, edo, use_round=True):
    float_steps = cents / OCTAVE_IN_CENTS * edo
    if use_round:
        return round(float_steps)
    return int(float_steps)


def ratio_to_steps(m, n, edo, use_round=True):
    return cents_to_steps(ratio_to_cents(m, n), edo, use_round=use_round)


def acoustic_scale(edo, use_round=True):
    # 7 note scale, kind-of approximating harmonics 7:8:9:10:11:12:13
    # NOTE/maybe FIXME: for 12edo the usual acoustic scale uses int instead of round
    # and there are actually some reasons for this,
    # explained in papers by Dmitri Tymoczko or Marek Žabka
    # but if we use round - we still get acoustic scale in 12edo,
    # just with a different "root"
    # for other edo scales this is not true
    scale = []
    for i in range(7, 14):
        scale.append(ratio_to_steps(i, 7, edo, use_round=use_round))
    return scale


# FIXME
# def parse_voicing_for_gestures():
#     multi_gesture = [
#         [{(0, -1): 2}] * 4,  # '0_:2x4' # '0_:8'
#         [{'.': 4}, {(3, -1): 2}, {(2, -1): 2}],  # '.:4 3_:2 2_:2'
#         *split([1, 2, 1, 3, 1, 2, 1, 3], zipped=False)
#     ]
#     for note_info in multi_gesture[voice]:
#         # FIXME: i assume here that gesture output is melody
#         if type(note_info) is dict:
#             pitch_info = list(note_info.keys())[0]
#             dur = list(note_info.values())[0]
#         else:
#             pitch_info = note_info
#             dur = 1
#         if type(pitch_info) is tuple:
#             note_idx, octave = pitch_info
#         else:
#             note_idx = pitch_info
#             octave = 0
#         if type(note_idx) is int:
#             note = chord[note_idx] + octave * edo
#         else:
#             note = note_idx
#         motif.append((note, dur))


def parse_voicing(voicing):
    # TODO/FIXME: create nice syntax for voicings
    ops = dict()
    # FIXME: find a better name
    chunks = voicing.split(' ')
    for chunk in chunks:
        octave = 0
        octave -= chunk.count('_')
        chunk = chunk.replace('_', '')
        octave += chunk.count('^')
        chunk = chunk.replace('^', '')
        ops[int(chunk)] = octave
    return ops


def apply_voicing(chord, voicing, octave):
    if voicing is None:
        return chord
    voicing_ops = parse_voicing(voicing)
    # FIXME: copy chord
    for voice_idx in voicing_ops:
        chord[voice_idx] += octave * voicing_ops[voice_idx]
    return chord


def build_tonic_steps(scale, voices_count, quality):
    # TODO: add support for octatonic scales
    # FIXME: in reality there should be some analysis of the scale
    # that corresponds to the quality
    # say, that we understand, that 2 steps really produce tertian quality
    # TODO: add support for higher limits (7-, 11-, 13-)
    # TODO: add support polytonality
    assert voices_count in [3, 4]
    steps = None
    if len(scale) == 7:
        if quality == 'tertian':
            steps = [2] * (voices_count - 1)
        elif quality == 'quartal':
            steps = [3] * (voices_count - 1)
        elif quality == 'add6':
            # FIXME: this is a bit ugly
            assert voices_count == 4
            steps = [2, 2, 1]
        else:
            assert quality == 'add2'
            assert voices_count == 4
            # FIXME: it's add9, really
            # also known as mu chord
            steps = [2, 2, 4]
    else:
        assert len(scale) == 10
        # TODO: add more support for decatonic scales
        # for 7-limit tetrads, the decatonic pattern is
        # X _ _ X _ _ X _ X _ X
        quality == 'tertian'
        steps = [3, 3, 2]
        steps = steps[:voices_count]
    return steps


# FIXME: rename voices to voices_count
# FIXME: hide defaults for quality and voices
def chord_in_scale(scale, degree, voices=3, quality='tertian'):
    # FIXME: this approach won't work in non-fifth based edos
    # and scales with length not equal to 7 or 10
    steps = build_tonic_steps(scale, voices, quality)

    assert 0 < degree <= len(scale)
    root = degree - 1

    chord = []
    sum_step = 0
    for i in range(voices):
        chord.append(root + sum_step)
        if i < voices - 1:
            sum_step += steps[i]
    return chord


def chord_in_edo(scale, edo, degree, voices=3, quality='tertian', voicing=None):
    chord = chord_in_scale(scale, degree=degree, voices=voices, quality=quality)
    if voices == 3:
        chord.append(chord[0] + len(scale))
    # FIXME: hacky default voicing, remove it
    if voicing is None:
        voicing = '0_'
    chord = apply_voicing(chord, voicing, octave=len(scale))
    # TODO: add option to sort notes, so that bass is note 0
    # NOTE/maybe FIXME: we switch here from scale to full edo, maybe there's a better approach
    return [note_in_scale(scale, n, edo) for n in chord]


# FIXME: inconsistent naming
# "in_edo" in "chord_in_edo" means something different
# than "in_edo" in "cadence_in_edo"
def cadence_in_edo(chord, new_chord_shape, edo, new_root=None, jump=False):
    # FIXME: should be a function/Chord() method
    new_pc_set = set()
    new_pc_set = frozenset([n % edo for n in new_chord_shape])
    if new_root is None:
        new_root = new_chord_shape[0] % edo

    # FIXME: right now we keep only 1 candidate for each possible pc set
    # but in the future we should keep a bit more promising candidates
    possible_new_sub_chords = dict()

    # FIXME: quartal chords don't have roots
    # TODO: for each voice we should respect voice range restrictions

    # FIXME: we also implicitly assume here that bass in voice 0
    # FIXME: maybe move this variable to optional parameters
    base_is_root = jump
    first_non_jump_note = 0
    # NOTE: this tolerance is in edo steps, not in scale steps
    # smooth_leading_tolerance_in_steps = round(edo / 3)  # FIXME: heuristic
    smooth_leading_tolerance_in_steps = round(edo * 5 / 12)  # other probable heuristic
    if base_is_root:
        first_non_jump_note = 1
        # TODO: inefficient
        for diff in range(smooth_leading_tolerance_in_steps + 1):
            # FIXME: we implicitly assume here that
            # we can have only 1 candidate within the tolerance
            # although with tolerance < edo/2 this should be always true
            signs = [-1, 1]
            if diff == 0:
                signs = [-1]
            for sign in signs:
                note = chord[0] + diff * sign
                pc = note % edo
                if pc == new_root:
                    assert (not possible_new_sub_chords)
                    # FIXME: ad hoc data structure
                    # tuple (chord, edo-step penalty)
                    possible_new_sub_chords[frozenset([pc])] = ([note], 0)
            if possible_new_sub_chords:
                break
        if not possible_new_sub_chords:
            candidates = []
            for sign in [-1, 1]:  # in current implementation the order here doesn't matter
                # TODO: inefficient, should be O(1)
                for diff in range(smooth_leading_tolerance_in_steps + 1, edo):
                    note = chord[0] + diff * sign
                    pc = note % edo
                    if pc == new_root:
                        candidates.append((note, pc))
                        break
            assert len(candidates) == 2
            note, pc = random.choice(candidates)
            possible_new_sub_chords[frozenset([pc])] = ([note], 0)
    else:
        possible_new_sub_chords[frozenset()] = ([], 0)

    # now find some voice-leading for "non-bass" notes
    for i in range(first_non_jump_note, len(chord)):
        possible_new_bigger_sub_chords = dict()
        found = False  # NOTE: actually it's not needed, we will always find something
        # but i will keep it for now because the code is complicated
        for diff in range(smooth_leading_tolerance_in_steps + 1):  # FIXME: probably inefficient for large edos
            signs = random.choice([[-1, 1], [1, -1]])  # FIXME: hack, to alleviate the problem with equal penalty
            if diff == 0:
                signs = [1]
            for sign in signs:
                note = chord[i] + diff * sign
                # FIXME: should be a function/Chord() method
                pc = note % edo
                if pc in new_pc_set:
                    found = True
                    penalty = diff
                    # FIXME: confusing naming
                    for prev_pc_set in possible_new_sub_chords:
                        prev_chord, prev_penalty = possible_new_sub_chords[prev_pc_set]
                        bigger_pc_set = frozenset(list(prev_pc_set) + [pc])
                        bigger_chord = copy(prev_chord) + [note]
                        bigger_penalty = penalty + prev_penalty
                        prev_bigger_penalty = None
                        if bigger_pc_set in possible_new_bigger_sub_chords:
                            prev_bigger_penalty = possible_new_bigger_sub_chords[bigger_pc_set][1]
                        if not prev_bigger_penalty or prev_bigger_penalty > bigger_penalty:
                            possible_new_bigger_sub_chords[bigger_pc_set] = (bigger_chord, bigger_penalty)
                        # FIXME: we could have an ambiguity here,
                        # if penalties are the same
        assert found
        possible_new_sub_chords = deepcopy(possible_new_bigger_sub_chords)

    # TODO: choose best chord
    if new_pc_set in possible_new_sub_chords:
        return possible_new_sub_chords[new_pc_set][0]
    new_chord = None
    best_pc_set_size = 0
    best_penalty = None
    for pc_set in possible_new_sub_chords:
        if len(pc_set) >= best_pc_set_size:
            candidate_chord, candidate_penalty = possible_new_sub_chords[pc_set]
            if (len(pc_set) > best_pc_set_size) or candidate_penalty >= best_penalty:
                # FIXME: we could have an ambiguity here,
                # if penalties are the same
                new_chord = candidate_chord
                best_penalty = candidate_penalty
                best_pc_set_size = len(pc_set)
    assert new_chord is not None

    # FIXME: with the current approach we can have some "problems",
    # e. g.:
    # - merging of 2 voices into the same note (should be easy to fix)
    # - voice crossings (should be easy to fix)
    # - not getting the full chord quality/modality (to fix this we would need to split notes)
    # although sometimes this is actually a desired behavior
    return new_chord


def cadence(chord, scale, edo, degree, voices=3, quality='tertian', jump=False):
    new_chord_shape = chord_in_scale(scale, degree=degree, voices=voices, quality=quality)
    # TODO: we switch here from scale to full edo, maybe there's a better approach
    new_chord_shape = [note_in_scale(scale, n, edo) for n in new_chord_shape]
    new_root = note_in_scale(scale, degree - 1, edo) % edo
    return cadence_in_edo(chord, new_chord_shape, edo, new_root, jump=jump)


def progress(scale, edo, degrees, voices_default=3, voicing=None, always_jump_on=[]):
    chords = []
    for info in degrees:
        # FIXME: this is ugly, need a parser
        if type(info) is tuple:
            degree, voices = info
        else:
            degree = info
            voices = voices_default

        jump = False
        quality = 'tertian'

        # parse degree and other properties
        if type(degree) is str:
            # FIXME: this is ugly, need a parser
            if 'q' in degree:
                quality = 'quartal'
                degree = degree.replace('q', '')
            elif 'add6' in degree:
                quality = 'add6'
                degree = degree.replace('add6', '')
            elif 'add2' in degree:
                quality = 'add2'
                degree = degree.replace('add2', '')
            if 'j' in degree:
                jump = True
                degree = degree.replace('j', '')
            if 'v4' in degree:
                voices = 4
                degree = degree.replace('v4', '')
            if 'v3' in degree:
                voices = 3
                degree = degree.replace('v3', '')
            degree = int(degree)

        # some additional fine-tuning
        if degree in always_jump_on:
            jump = True

        if not chords:
            chords.append(chord_in_edo(scale=scale, edo=edo, degree=degree, voices=voices,
                                       quality=quality, voicing=voicing))
        else:
            chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=degree, voices=voices,
                                  jump=jump, quality=quality))
    return chords


In [177]:
# V-I cadence

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'piano2')
set_synths()

edo = 12
scale = [0, 2, 4, 5, 7, 9, 11]  # 0/C diatonic/ionian
# scale = [0, 2, 3, 5, 7, 9, 10]  # 0/C dorian
# scale = [0, 2, 4, 6, 7, 9, 11]  # 0/C lydian
# scale = [0, 2, 4, 6, 7, 9, 10]  # 0/C acoustic
# scale = [0, 1, 3, 4, 6, 8, 10]  # 0/C altered

# jazz ii-V-I
# chords = progress(scale, edo, [
#     (2, 4),
#     (5, 4),
#     ('1j', 4)
# ])

# V7-I
# chords = progress(scale, edo, [(5, 4), '1j'])

# exploring Kostka-Payne chart
# https://i.stack.imgur.com/JfJbu.png
# iii -> vi -> S:[IV -> ii] -> D:[V <-> vii] -> I .
# (or maybe just vii -sometimes> V)
# iii -> S -> I
# D -> vi .
chords = progress(scale, edo,
    # [6, 4, (5, 4), 1]
    # [4, (5, 4), (5, 4), (5, 4), 1]
    # [6, 2, (5, 4), '1j']
    ['6add2v4', '4add6v4', '7v4', '1']
    , voicing='1_'
)

# plagal cadence
# chords = progress(scale, edo, [
#     (7, 4),
#     (4, 4),
#     ('1j', 4)
# ])

print(chords)
dur = 0.5
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       dur=dur, sus=dur * 0.95)


[[9, 0, 16, 23], [9, 2, 17, 24], [9, -1, 17, 26], [7, 0, 16, 28]]


In [174]:
# 22edo

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'piano2')
set_synths()

edo = 22
scale = build_scale_from_steps('3334333', edo=22, start=-4)  # porcupine[3|3]
# scale = [0, 4, 7, 10, 13, 16, 19]  # 0/C porcupine[6|0]
# scale = [x - 4 for x in acoustic_scale(edo, use_round=False)]
# scale = [x - 4 for x in acoustic_scale(edo, use_round=True)]

# exploring Kostka-Payne chart
# iii -> vi -> S:[IV -> ii] -> D:[V <-> vii] -> I .
# iii -> S -> I
# D -> vi .
chords = progress(scale, edo,
    # ['6v4', 4, 5, '1jv3']
    ['6qv4', 4, 5, '1jv3']
    # [4, 2, 5, 3, 6, 4, 7, 5, '1jv3']
    , voicing='0_ 2_ 3^'
    , voices_default=4
)

print(chords)
dur = 0.75
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       dur=dur, sus=dur * 0.95)


[[-10, 21, 9, 62], [-10, 24, 5, 62], [-13, 21, 5, 59], [-4, 24, 9, 62]]


In [137]:
# 17edo

from domblar.transformations import transpose

edo = 17
scale = [0, 3, 5, 8, 10, 13, 15]  # 3232323 # this scale works
scale = transpose(scale, edo)
# scale = [0, 3, 6, 7, 10, 13, 16]  # 3313331
# scale = [x - 4 for x in acoustic_scale(edo, use_round=True)]

# exploring Kostka-Payne chart
# iii -> vi -> S:[IV -> ii] -> D:[V <-> vii] -> I .
# iii -> S -> I
# D -> vi .
chords = progress(scale, edo,
    # [4, 7, '1j']
    # [6, 4, 5, '1j']
    [4, 2, 5, 3, 6, 4, 7, 5, '1jv3']
    , voices_default=4
    , voicing='0_'
)

print(chords)
dur = 0.75
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       dur=dur, sus=dur * 0.95)


[[8, 30, 34, 39], [3, 30, 34, 42], [3, 27, 32, 42], [3, 22, 32, 44], [0, 22, 30, 44], [0, 25, 30, 39], [-2, 25, 30, 37], [-7, 25, 32, 37], [0, 27, 34, 39]]


In [139]:
# part A
# in the spirit of Sergei Zagny - Sonata

import time

from domblar.edo import get_freq
from domblar.transformations import split

def set_synths():
    d.set_synth(0, '*oelfass')
    d.set_synth(1, 'celeste')
    d.set_synth(2, 'celeste')
    d.set_synth(3, 'big_grand')
set_synths()

edo = 22
scale = [x - 4 for x in [0, 3, 6, 9, 13, 16, 19]]  # porcupine[3|3]
# scale = [x - 4 for x in [0, 4, 7, 10, 13, 16, 19]]  # porcupine[6|0]
# scale = [x - 4 for x in [0, 3, 7, 10, 13, 16, 19]]  # porcupine[5|1]

# edo = 12
# scale = [x - 2 for x in [0, 2, 4, 5, 7, 9, 11]]  # 10/Bb diatonic/ionian
# scale = [x - 2 for x in acoustic_scale(edo, use_round=True)]

# edo = 17
# scale = [x - 2 for x in acoustic_scale(edo, use_round=True)]
# scale = [x - 3 for x in [0, 3, 5, 8, 10, 13, 15]]  # 3232323
# # scale = [0, 3, 6, 7, 10, 13, 16]  # 3313331

# edo = 15
# scale = [x - 2 for x in acoustic_scale(edo, use_round=True)]
# scale = [0, 1, 4, 5, 8, 9, 12]  # 1313133

progression0 = ['1j'] * 7
progression1 = ['4j'] + cycle(sequence(1, [2, -3], mod=len(scale), notes=15),
                              dup_start=True, dup_end=False)
chords = progress(scale, edo,
    progression0 + progression1 + progression0, voicing='0_ 1_'
    , voices_default=4
)

# FIXME: deprecated
# def apply_multi_gesture(chords, multi_gesture, edo):
#     result = [[] for _ in range(len(multi_gesture))]
#     for chord in chords:
#         for voice in range(len(multi_gesture)):
#             motif = []
#             # FIXME: move to parse_voicing
#             for note_info in multi_gesture[voice]:
#                 # FIXME: i assume here that gesture output is melody
#                 if type(note_info) is dict:
#                     pitch_info = list(note_info.keys())[0]
#                     dur = list(note_info.values())[0]
#                 else:
#                     pitch_info = note_info
#                     dur = 1
#                 if type(pitch_info) is tuple:
#                     note_idx, octave = pitch_info
#                 else:
#                     note_idx = pitch_info
#                     octave = 0
#                 if type(note_idx) is int:
#                     note = chord[note_idx] + octave * edo
#                 else:
#                     note = note_idx
#                 motif.append((note, dur))
#             result[voice].extend(motif)
#     return result


def play_events(voices, edo, client, synth_idx=None,
                dur=0.25, sus=None,
                rep=1,
                voice_amps=[]
                ):
    if synth_idx is None:
        synth_idx = list(range(len(voices)))
    if type(rep) is list:
        assert len(synth_idx) == len(rep)
    else:
        rep = [rep] * len(synth_idx)
    last_reps = [0] * len(synth_idx)

    scale = list(range(edo))

    last_event_idx = [0] * len(voices)
    last_event_time = [0] * len(voices)
    start_time = time.time()
    latency = 0.02  # TODO: move to client, or to some class, or to params
    beats_ahead = 2

    progressed = True
    while progressed or waiting:
        progressed = False
        waiting = False
        for voice_idx in range(len(voices)):
            if last_event_idx[voice_idx] >= len(voices[voice_idx]):
                continue
            waiting = True
            cur_beat = (time.time() - start_time) / dur
            if last_event_time[voice_idx] > cur_beat + beats_ahead:
                continue

            note, note_beat_count = voices[voice_idx][last_event_idx[voice_idx]]
            freq = get_freq(note, scale, edo)
            send_note_dur = dur
            if sus:
                send_note_dur = sus
            send_note_dur *= note_beat_count
            amp = 1.0
            if voice_amps:
                amp *= voice_amps[voice_idx]

            cur_beat = (time.time() - start_time) / dur
            if last_event_time[voice_idx] > cur_beat + beats_ahead:
                continue
            timetag = start_time + last_event_time[voice_idx] * dur + latency

            client.send_note(
                synth_idx[voice_idx] + last_reps[voice_idx],
                freq=freq, dur=send_note_dur, amp=amp,
                timetag=timetag, channel=0)  # TODO: for MPE with channels use channel=note_idx
            last_reps[voice_idx] = (last_reps[voice_idx] + 1) % rep[voice_idx]

            last_event_idx[voice_idx] += 1
            last_event_time[voice_idx] += note_beat_count
            progressed = True

        if waiting and not progressed:
            # sleep 1 beat
            time.sleep(dur)

# print(chords)
# dur = 0.3
# d.play(chords, list(range(edo)), edo,
#        synth_idx=[0, 1, 2, 3],
#        dur=dur, sus=dur * 0.95)

gesture = [
    '0_:4x2',
    '.:4 3_:2 2_:2',
    *split([1, 1, 2, 3] * 2),
    # *split([2, 1, 2, 3] * 2, zipped=False),
    # *[[{i: 2}] for i in range(4)],
]
voices = apply_gesture(chords, gesture, edo)

dur = 0.15/1.2
play_events(
    voices=voices, edo=edo, client=d.client,
    dur=dur, sus=dur * 0.95,
    voice_amps=[1.2, 0.9, 0.7, 0.7]
)


KeyboardInterrupt: 

In [143]:
# part B
# NRT-like progressions + L-systems
# also in the spirit of Sergei Zagny - Sonata

from collections import defaultdict

from domblar.transformations import shift

def set_synths():
    for i in range(4):
        d.set_synth(i, 'big_grand')
    d.set_synth(2, '*vibration')
    # d.set_synth(4, '*vibration')
set_synths()


# FIXME: unused, but helpful
def stats(s):
    counts = defaultdict(int)
    max_runs = defaultdict(int)
    cur_run = 0
    prev_char = ''
    for c in s:
        counts[c] += 1
        if c == prev_char:
            cur_run += 1
        else:
            if prev_char:
                max_runs[prev_char] = max(max_runs[prev_char], cur_run)
            cur_run = 1
        prev_char = c
    max_runs[prev_char] = max(max_runs[prev_char], cur_run)
    return {'counts': counts.items(), 'max_runs': max_runs.items()}

# FIXME: rename
def run_l_system(rules, start, iterations):
    # FIXME
    # print('0: ', start)
    cur = start
    for c in list(rules.keys()):
        if type(rules[c]) is not list:
            rules[c] = [rules[c]]
    # FIXME: unused variable
    for i in range(iterations):
        c_counter = defaultdict(int)
        rewritten = ''
        for c in cur:
            rewritten += rules[c][c_counter[c] % len(rules[c])]
            c_counter[c] += 1
        # FIXME
        # print(f'{i + 1}: ', rewritten.startswith(cur), stats(rewritten))
        cur = rewritten
    return cur

rules = {
    '1': ['12', '112'],
    '2': ['221', '21']
}
# NOTE: this rule is also prefix-preserving
# rules = {
#     '1': '12',
#     '2': '1'
# }
l_rhythm = run_l_system(rules, '2', 9)


# Tonnetz progressions
# PLR group

from domblar.transformations import transpose
from domblar.music_math import ratio_to_floats

edo = 15
steps = '3131331'
steps = [int(x) for x in steps]
scale = build_scale_from_steps(steps, edo)  # Hanson/Keemun/Orgone[7] modes

# TODO: The basic "major" triad is set to 7:9:11
# and the "minor" triad set to its inverse.
# edo = 11
# steps = '2212121'  # Orgone[7] modes

# print('debug', -round(edo * ratio_to_floats(6, 5)), round(edo * ratio_to_floats(5, 4)))

def build_chord_loop(scale, edo):
    chord0 = chord_in_edo(scale, edo, 1,
                          voices=4, quality='tertian')
    chord1 = chord_in_edo(transpose(scale, -round(edo * ratio_to_floats(6, 5))), edo, 1,
                          voices=4, quality='add6')
    chord2 = chord_in_edo(transpose(scale, round(edo * ratio_to_floats(5, 4))), edo, 1,
                          voices=4, quality='tertian')

    chord_loop = cycle([chord1, chord0, chord2],
                       dup_start=False, dup_end=False)
    return chord_loop

chord_loop = build_chord_loop(scale, edo)

# build chords
loop_count = 7
chords = [chord_loop[-1]]
for _ in range(loop_count):
    for _ in range(3):
        for rep in range(3):
            for chord in chord_loop:
                chords.append(cadence_in_edo(chords[-1], chord, edo))
        steps = shift(steps, 1)
        steps = [int(x) for x in steps]
        scale = build_scale_from_steps(steps, edo)
        chord_loop = build_chord_loop(scale, edo)
for chord in chord_loop[:2]:
    chords.append(cadence_in_edo(chords[-1], chord, edo))

muls = [1]
r_idx = 0
for _ in range(loop_count):
    sub_muls = []
    for rep in range(3):
        for chord in chord_loop:
            sub_muls.append(int(l_rhythm[r_idx]))
            r_idx += 1
    muls.extend(sub_muls * 3)
    print(sub_muls)
muls.extend([15, 16])

chords = chords[:len(muls)]
print(chords[-4:])

# debug_end_count = 10
# chords = chords[-debug_end_count:]
# muls = muls[-debug_end_count:]

# TODO: duplicate voice for 2 timbres
dur = 0.25
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       # synth_idx=[0, 1, 2, [3, 4]],
       dur=dur, sus=dur * 0.7,
       muls=muls,
       # FIXME: increase amp for "*vibration"
       voice_amps=[1, 0.6, 1.3, 0.8])


[2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1]
[2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1]
[2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2]
[1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1]
[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2]
[2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2]
[2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1]
[[-2, 5, 24, 32], [-3, 4, 23, 30], [-4, 4, 22, 30], [-1, 4, 23, 30]]


KeyboardInterrupt: 

In [193]:
# part A'
# decatonic scales
# 22edo, 15edo

def set_synths():
    # d.set_synth(0, 'big_grand')
    # d.set_synth(1, 'big_grand')
    # d.set_synth(2, 'celeste')
    # d.set_synth(3, '*oelfass')
    d.set_synth(0, 'celeste')
    d.set_synth(1, 'celeste')
    d.set_synth(2, 'celeste')
    d.set_synth(3, '*vibration')
set_synths()

edo = 22
scale = build_scale_from_steps('2232222322', edo=edo, start=-edo)  # pajara[10] symmetric decatonic
# scale = build_scale_from_steps('3133131331', edo=edo, start=-edo)  # astrology[10] mode

# edo = 15
# scale = build_scale_from_steps('2121212112', edo=edo, start=-edo)  # Blackwood decatonic

# edo = 17
# scale = build_scale_from_steps('2122122212', edo=edo, start=-edo)  # neutrominant

# edo = 12
# # Four-semitone tritone scale/Messiaen's 7th "mode"
# scale = build_scale_from_steps('1121111211', edo=edo, start=-edo)

# TODO:
# A decatonic scale that has been used or considered by Kyle Gann and La Monte Young
# in 13-limit just intonation is
# 1/1, 12/11, 32/27, 9/7, 4/3, 132/91, 3/2, 18/11, 16/9, 176/91, and 2/1.

# chords = progress(scale, edo,
#     # cycle(['2', '7', '3', '9', '4', '8', '1j']) * 8 + ['1j']
#     # [4, 2, 5, 3, 6, 4, 7, 5, '1j'] * 10
#     ['5', '7', '1j']
#     , voicing='0__ 2_'
#     , voices_default=4
# )

progression = cycle(sequence(5, [-3,-4], mod=len(scale), notes=63),
                    dup_start=True) + ['1j']
progression = cycle(sequence(5, [1,-2,-1,1], mod=len(scale), notes=63),
                    dup_start=True) + ['1j']
muls_for_3rd_part = []
stop_dur = 6
last_dur = 1
for idx, deg in enumerate(progression):
    if deg == 1:
        muls_for_3rd_part.append(stop_dur)
        last_dur = stop_dur
        # stop_dur += 2
    else:
        muls_for_3rd_part.append(last_dur)
        last_dur = max(last_dur // 2, 1)
muls_for_3rd_part[-1] = stop_dur * 2

chords = progress(
    scale, edo, progression,
    # voicing='0__ 2_',
    voicing='0__ 2_',
    voices_default=4,
    always_jump_on=[1],
)
# print(chords)
print(progression)

# dur = 0.08212/1.2
dur = 0.15
print(dur)

d.play(
    chords, list(range(edo)), edo,
    synth_idx=[0, 1, 2, 3],
    dur=dur, sus=dur * 0.95,
    muls=muls_for_3rd_part,
    # mul=muls[:len(chords)],
)


[5, 6, 4, 3, 4, 5, 3, 2, 3, 4, 2, 1, 2, 3, 1, 10, 1, 2, 10, 9, 10, 1, 9, 8, 9, 10, 8, 7, 8, 9, 7, 6, 7, 8, 6, 5, 6, 7, 5, 4, 5, 6, 4, 3, 4, 5, 3, 2, 3, 4, 2, 1, 2, 3, 1, 10, 1, 2, 10, 9, 10, 1, 9, 1, 10, 9, 10, 2, 1, 10, 1, 3, 2, 1, 2, 4, 3, 2, 3, 5, 4, 3, 4, 6, 5, 4, 5, 7, 6, 5, 6, 8, 7, 6, 7, 9, 8, 7, 8, 10, 9, 8, 9, 1, 10, 9, 10, 2, 1, 10, 1, 3, 2, 1, 2, 4, 3, 2, 3, 5, 4, 3, 4, 6, 5, '1j']
0.15


In [ ]:
# more microtonal scales

# # edo = 26
# # scale = [x - 5 for x in [0, 5, 7, 12, 14, 19, 21]]  # 21/Bb LsLsLsL/5252525
# # scale = [x - 5 for x in [0, 5, 7, 12, 17, 19, 24]]  # 21/Bb LsLLsLs/5255252
# # scale = [x - 5 for x in [0, 5, 10, 12, 17, 19, 24]]  # 21/Bb LLsLsLs/5525252

In [ ]:
# V/V - V - I

def set_synths():
    for i in range(4):
        d.set_synth(i, '*vibration')
set_synths()

edo = 12
scale = [0, 2, 4, 5, 7, 9, 11]  # 0/C diatonic
# scale = [0, 2, 4, 6, 7, 9, 11]  # 0/C lydian
# scale = acoustic_scale(edo)  # [0, 2, 4, 6, 7, 9, 10], 0/C acoustic

# edo = 22
# scale = [0, 4, 7, 10, 13, 16, 19]  # 0/C porcupine[6|0]

scale_v = [x - edo + scale[4] for x in scale]  # 7/G same scale
scale_vv = [x + scale_v[4] for x in scale_v]  # 2/D same scale

chords = []

# V/V - V - I
chords.append(chord_in_edo(scale=scale_v, edo=edo, degree=5, voices=4))
# chords.append(chord_in_edo(scale=scale_vv, edo=edo, degree=7, voices=4))
# chords.append(cadence(chords[-1], scale=scale_v, edo=edo, degree=5, voices=4))
chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=5, voices=4))
chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=1, voices=3, jump=True))

# chords.append(cadence(chords[-1], scale=scale_vv, edo=edo, degree=5, voices=4))
# # chords.append(chord_in_edo(scale=scale_v, edo=edo, degree=5, voices=4))
# chords.append(cadence(chords[-1], scale=scale_v, edo=edo, degree=7, voices=4))
# chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=5, voices=4))
# chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=1, voices=3, jump=True))

print(chords)
dur = 0.75
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       dur=dur, sus=dur * 0.95)


[[2, -6, 9, 12], [2, -7, 7, 11], [0, -8, 7, 12]]


In [ ]:
# secondary plagal
# IV/IV - IV - I

edo = 12
scale = [0, 2, 4, 5, 7, 9, 11]  # 0/C diatonic
# # scale = [0, 2, 4, 6, 7, 9, 11]  # 0/C lydian
# # scale = [0, 2, 4, 6, 7, 9, 10]  # 0/C acoustic

# edo = 22
# scale = [0, 4, 7, 10, 13, 16, 19]  # 0/C porcupine[6|0]

scale_iv = [x - edo + scale[3] for x in scale]  # 7/G same scale
# scale_iviv = [x + scale_v[4] for x in scale_v]  # 2/D same scale

chords = []

# IV/IV - IV - I
chords.append(chord_in_edo(scale=scale_iv, edo=edo, degree=4, voices=4))
chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=4, voices=4))
# chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=5, voices=4))
chords.append(cadence(chords[-1], scale=scale, edo=edo, degree=1, voices=3, jump=True))

print(chords)
dur = 0.5
d.play(chords, list(range(edo)), edo,
       synth_idx=[0, 1, 2, 3],
       dur=dur, sus=dur * 0.95)


[[-2, -10, 5, 9], [0, -8, 5, 9], [0, -8, 4, 7]]


In [132]:
# FIXME: outdated
# FIXME: replace with method/function
finetuning = 0

try:
    finetuning
except Exception as e:
    d.open_editor(0)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(0)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(0)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            d.load_preset(i)
    else:
        d.print_params(0)

In [ ]:
# cleanup
d.stop_server()